In [83]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

***Projected Subgradient Method***

In [84]:
def read_data(verbose=False):

    data = pd.read_csv("all_stocks_5yr.csv")
    df = data[["date", "Name", "close"]]

    prices = df.pivot(index="date", columns="Name", values="close")
    prices = prices.dropna(axis=1)

    # with open("prices.txt", "w") as f:
    #     f.write(prices.to_string())
    #     f.close()

    returns = prices.pct_change().dropna()
    # with open("rendements.txt", "w") as f:
    #     f.write(returns.to_string())
    #     f.close()


    X = returns.values
    mu = returns.mean().values
    Sigma = returns.cov().values


    if verbose:
        print("X shape =", X.shape)
        print("mu shape =", mu.shape)
        print("Sigma shape =", Sigma.shape)
    
    return X, mu, Sigma



In [87]:
def projection_on_simplex(v):
    """
    Project a vector v onto the simplex:
        Δ = { w >= 0, sum(w) = 1 }
    """
   
    u = sorted(v, reverse=True)
    cumulative_sum = np.cumsum(u)

    thetas = (cumulative_sum - 1) / (np.arange(1, len(v) + 1))
    k = np.where(u - thetas > 0)[0].max()

    theta = (cumulative_sum[k] - 1) / (k + 1)
    w = np.maximum(v - theta, 0)

    return w

def test(): 
    v = np.array([0.9, 0.9, 0.2])
    v2 = -1 * v

    print("w1 : ", projection_on_simplex(v))
    print("w2 : ", projection_on_simplex(v2))

#test()

def sign(v):
    result = np.zeros_like(v)

    result[v > 0] = 1
    result[v < 0] = -1

    return result


def obj_f_nonSmooth(w, wprev, Sigma, mu, lam, c=0.0):

    return 0.5 * w.T @ Sigma @ w -lam * (mu @ w) + \
            c * np.sum(np.abs(w - wprev))

def projected_subgradient_method(lam, c, Sigma, mu, wprev, alpha0=1, max_iter=50000, tol= 1e-8, plot=False): 

    w = wprev
    k = 0
    if plot:
        obj_values = []
    for i in tqdm(range(max_iter)):

        if plot:
            obj_values.append(obj_f_nonSmooth(w, wprev, Sigma, mu, lam, c))

        g = Sigma @ w - lam * mu + c * np.sign(w - wprev)
        step_size = alpha0 / (k + 1)
        w_new = projection_on_simplex(w - step_size * g)
        
        if (np.linalg.norm(w_new - w) < tol):
            print(f"Converged in {i} iterations.")
            break

        w = w_new


    if plot:
        plt.plot(obj_values)
        plt.xlabel("Iteration")
        
        plt.yscale("log")
        plt.ylabel("Objective value (log scale)")
        plt.title("Convergence of Projected Subgradient Method")
        plt.show()
    
    return w




    

In [ ]:
X, mu, Sigma = read_data(verbose=True)
lam = 0.1
c = 1
wprev = np.ones(X.shape[1]) / X.shape[1]

w = projected_subgradient_method(lam, c, Sigma, mu, wprev, tol= 1e-8, plot=True)
print("Optimal weights:", w)
print("Sum of weights:", np.sum(w))


X shape = (1258, 470)
mu shape = (470,)
Sigma shape = (470, 470)


100%|██████████| 50000/50000 [00:12<00:00, 4165.81it/s]

Optimal weights: [0.00427979 0.         0.00429547 0.         0.         0.
 0.00429645 0.         0.         0.00429603 0.00428698 0.
 0.00428781 0.         0.         0.         0.00425647 0.
 0.         0.         0.00428512 0.00429723 0.         0.
 0.00429936 0.00429716 0.         0.         0.         0.00426542
 0.         0.         0.00429652 0.00424836 0.         0.00430148
 0.         0.         0.         0.         0.         0.
 0.         0.00420655 0.004229   0.00430173 0.         0.
 0.0042527  0.         0.00429323 0.         0.         0.
 0.00429172 0.         0.         0.         0.         0.00427653
 0.00429725 0.         0.         0.00422322 0.         0.
 0.00429947 0.00429904 0.         0.         0.         0.00426934
 0.00427344 0.00428104 0.00427891 0.00428315 0.00428829 0.00429488
 0.         0.00428122 0.         0.00427353 0.         0.00429713
 0.         0.00429577 0.00428567 0.00426556 0.         0.00414876
 0.         0.         0.         0.004298